# Classification_Airline Delay
#### 2024.7.2

### Purpose of Data Analysis
    - Predict how much an aircraft will be delayed upon arrival.
    - Identify the reasons for delays and address these issues to improve service quality.
### Key Points
    - Learn the details of data processing, including join operations, handling missing values, and removing columns.
    - Learn methods to analyze categorical and numeric data (utilizing pandas profiling for easy EDA).
    -  Learn how to use XGBoost, analyze feature importance, and consider additional analyses.

In [43]:
pip install ydata-profiling

Note: you may need to restart the kernel to use updated packages.


In [44]:
# library for feature engineering and EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from IPython.display import Image
from ydata_profiling import ProfileReport
from datetime import datetime

# library for statistic
from scipy import stats
from scipy.stats import kruskal
from scipy.stats import f_oneway

# library for machine learning
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

%matplotlib inline

## 1. Data Processing 1 

### 1-1 Data Load 

In [45]:
random_state = 1234

df_airlines = pd.read_csv("./data/4.Classification_Airline_delay/airlines.csv")
df_flights = pd.read_csv("./data/4.Classification_Airline_delay/flights.csv")
df_tzone_meta = pd.read_csv("./data/4.Classification_Airline_delay/tzone_meta.csv")
df_airports = pd.read_csv("./data/4.Classification_Airline_delay/airports.csv")
df_planes = pd.read_csv("./data/4.Classification_Airline_delay/planes.csv")
df_weather = pd.read_csv("./data/4.Classification_Airline_delay/weather.csv")

Check all the data

In [46]:
df_airlines.head()

,carrier,name
0,AA,American Airlines Inc.
1,AS,Alaska Airlines Inc.
2,B6,JetBlue Airways
3,DL,Delta Air Lines Inc.
4,EV,ExpressJet Airlines Inc.


In [47]:
df_flights.head()
# time needs to be change in proper manner

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,1537,N54711,EWR,MIA,148.0,1085.0,21.0,14.0,2017-10-01 21:00:00
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,1161,N37287,EWR,IAH,183.0,1400.0,5.0,25.0,2017-10-01 05:00:00
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,27,N547JB,EWR,MCO,133.0,937.0,5.0,45.0,2017-10-01 05:00:00
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,761,N38446,LGA,ORD,108.0,733.0,5.0,50.0,2017-10-01 05:00:00
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,521,N675NK,EWR,FLL,156.0,1065.0,6.0,0.0,2017-10-01 06:00:00


In [48]:
df_flights.info()
# some missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303748 entries, 0 to 303747
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            303748 non-null  int64  
 1   month           303748 non-null  int64  
 2   day             303748 non-null  int64  
 3   dep_time        295938 non-null  float64
 4   sched_dep_time  303748 non-null  int64  
 5   dep_delay       295934 non-null  float64
 6   arr_time        295600 non-null  float64
 7   sched_arr_time  303748 non-null  int64  
 8   arr_delay       294896 non-null  float64
 9   carrier         303748 non-null  object 
 10  flight          303748 non-null  int64  
 11  tailnum         302757 non-null  object 
 12  origin          303748 non-null  object 
 13  dest            303748 non-null  object 
 14  air_time        294896 non-null  float64
 15  distance        303748 non-null  float64
 16  hour            303748 non-null  float64
 17  minute    

In [49]:
df_flights.isna().sum()
# our target is predicting arr_deay, so will drop these NA

year                 0
month                0
day                  0
dep_time          7810
sched_dep_time       0
dep_delay         7814
arr_time          8148
sched_arr_time       0
arr_delay         8852
carrier              0
flight               0
tailnum            991
origin               0
dest                 0
air_time          8852
distance             0
hour                 0
minute               0
time_hour            0
dtype: int64

In [50]:
df_flights = df_flights.dropna(subset=['arr_delay']).copy()

In [51]:
df_tzone_meta.head()

,tzone,tzone_name
0,1,America/New_York
1,2,America/Chicago
2,3,America/Denver
3,4,America/Phoenix
4,5,America/Anchorage


In [52]:
df_airports.head()

,faa,name,lat,lon,alt,tz,dst,tzone
0,0V4,Brookneal/Campbell County Airport,37.141701,-79.016403,596,-5.0,A,1.0
1,19S,Sublette Municipal Airport,37.491402,-100.830002,2908,-5.0,A,NaN
2,23M,Clarke County Airport,32.084900,-88.738899,320,-6.0,A,2.0
3,2A5,Causey Airport,35.911800,-79.617599,723,-5.0,A,1.0
4,2H0,Shelby County Airport,39.410400,-88.845398,618,-6.0,A,2.0


In [53]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   faa     1333 non-null   object 
 1   name    1333 non-null   object 
 2   lat     1333 non-null   float64
 3   lon     1333 non-null   float64
 4   alt     1333 non-null   int64  
 5   tz      1277 non-null   float64
 6   dst     1277 non-null   object 
 7   tzone   1199 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 83.4+ KB


In [54]:
# check if df_flights and fd_airport can be join
# 'dest' 'origin
print(len(df_airports.faa.unique()))
df_airports.faa.unique()

1333


array(['0V4', '19S', '23M', ..., 'ZSY', 'ZUN', 'ZZV'], dtype=object)

In [55]:
np.intersect1d(df_flights.dest.unique(), df_airports.faa.unique())

array(['ABQ', 'ACK', 'ALB', 'ANC', 'ATL', 'AUS', 'AVL', 'AVP', 'BDL',
       'BGR', 'BHM', 'BNA', 'BOS', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN',
       'CAE', 'CAK', 'CHA', 'CHO', 'CHS', 'CLE', 'CLT', 'CMH', 'CVG',
       'DAB', 'DAL', 'DAY', 'DCA', 'DEN', 'DFW', 'DSM', 'DTW', 'EGE',
       'FLL', 'FNT', 'FWA', 'GRR', 'GSO', 'GSP', 'HDN', 'HNL', 'HOU',
       'HYA', 'IAD', 'IAH', 'ILM', 'IND', 'JAC', 'JAX', 'LAS', 'LAX',
       'LEX', 'LGB', 'MCI', 'MCO', 'MDW', 'MEM', 'MHT', 'MIA', 'MKE',
       'MSN', 'MSP', 'MSY', 'MTJ', 'MVY', 'MYR', 'OAK', 'OMA', 'ORD',
       'ORF', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PSP', 'PVD', 'PWM',
       'RDU', 'RIC', 'RNO', 'ROC', 'RSW', 'SAN', 'SAT', 'SAV', 'SBN',
       'SDF', 'SEA', 'SFO', 'SJC', 'SLC', 'SMF', 'SNA', 'SRQ', 'STL',
       'SYR', 'TPA', 'TUS', 'TVC', 'TYS', 'XNA'], dtype=object)

In [56]:
np.intersect1d(df_flights.origin.unique(), df_airports.faa.unique())

array(['EWR', 'JFK', 'LGA'], dtype=object)

In [57]:
df_planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102AA,2007.0,Fixed wing single engine,AVIAT AIRCRAFT INC,A-1B,1,2.0,NaN,Reciprocating
1,N103AA,2007.0,Fixed wing single engine,AVIAT AIRCRAFT INC,A-1B,1,2.0,NaN,Reciprocating
2,N103SY,2014.0,Fixed wing multi engine,EMBRAER S A,ERJ 170-200 LR,2,88.0,NaN,Turbo-fan
3,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55.0,NaN,Turbo-fan
4,N105AA,1978.0,Fixed wing single engine,CESSNA,R182,1,4.0,111.0,Reciprocating


In [58]:
df_planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3521 entries, 0 to 3520
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   tailnum       3521 non-null   object 
 1   year          3419 non-null   float64
 2   type          3521 non-null   object 
 3   manufacturer  3521 non-null   object 
 4   model         3521 non-null   object 
 5   engines       3521 non-null   int64  
 6   seats         3511 non-null   float64
 7   speed         62 non-null     float64
 8   engine        3521 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 247.7+ KB


In [59]:
df_planes.isna().sum()

tailnum            0
year             102
type               0
manufacturer       0
model              0
engines            0
seats             10
speed           3459
engine             0
dtype: int64

In [60]:
df_planes=df_planes.drop('speed',axis=1)

In [61]:
df_weather.head()


,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2017.0,1.0,1,0,NaN,NaN,NaN,200.0,17.26170,19.864419,NaN,NaN,10.0,2017-01-01 08:00:00
1,EWR,2017.0,1.0,1,1,NaN,NaN,NaN,220.0,16.11092,18.540125,NaN,NaN,10.0,2017-01-01 09:00:00
2,EWR,2017.0,1.0,1,2,NaN,NaN,NaN,190.0,11.50780,13.242946,NaN,NaN,10.0,2017-01-01 10:00:00
3,EWR,2017.0,1.0,1,3,NaN,NaN,NaN,190.0,8.05546,9.270062,NaN,NaN,10.0,2017-01-01 11:00:00
4,EWR,2017.0,1.0,1,4,NaN,NaN,NaN,220.0,9.20624,10.594357,NaN,NaN,10.0,2017-01-01 12:00:00


In [62]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26201 entries, 0 to 26200
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   origin      26201 non-null  object 
 1   year        26201 non-null  float64
 2   month       26201 non-null  float64
 3   day         26201 non-null  int64  
 4   hour        26201 non-null  int64  
 5   temp        559 non-null    float64
 6   dewp        559 non-null    float64
 7   humid       559 non-null    float64
 8   wind_dir    23540 non-null  float64
 9   wind_speed  23554 non-null  float64
 10  wind_gust   23554 non-null  float64
 11  precip      1370 non-null   float64
 12  pressure    433 non-null    float64
 13  visib       26190 non-null  float64
 14  time_hour   26201 non-null  object 
dtypes: float64(11), int64(2), object(2)
memory usage: 3.0+ MB


In [63]:
df_weather.isna().sum()

origin            0
year              0
month             0
day               0
hour              0
temp          25642
dewp          25642
humid         25642
wind_dir       2661
wind_speed     2647
wind_gust      2647
precip        24831
pressure      25768
visib            11
time_hour         0
dtype: int64

In [64]:
df_weather = df_weather.drop(["temp", "dewp", "humid","precip","pressure"], axis=1)
df_weather

,origin,year,month,day,hour,wind_dir,wind_speed,wind_gust,visib,time_hour
0,EWR,2017.0,1.0,1,0,200.0,17.26170,19.864419,10.0,2017-01-01 08:00:00
1,EWR,2017.0,1.0,1,1,220.0,16.11092,18.540125,10.0,2017-01-01 09:00:00
2,EWR,2017.0,1.0,1,2,190.0,11.50780,13.242946,10.0,2017-01-01 10:00:00
3,EWR,2017.0,1.0,1,3,190.0,8.05546,9.270062,10.0,2017-01-01 11:00:00
4,EWR,2017.0,1.0,1,4,220.0,9.20624,10.594357,10.0,2017-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...
26196,LGA,2017.0,12.0,30,19,210.0,9.20624,10.594357,5.0,2017-12-31 03:00:00
26197,LGA,2017.0,12.0,30,20,200.0,10.35702,11.918651,10.0,2017-12-31 04:00:00
26198,LGA,2017.0,12.0,30,21,280.0,8.05546,9.270062,10.0,2017-12-31 05:00:00
26199,LGA,2017.0,12.0,30,22,260.0,10.35702,11.918651,10.0,2017-12-31 06:00:00


Since wheather would be affected by weather from yesterday, i will use ffill.

In [65]:
df_weather = df_weather.sort_values(["origin", "time_hour"]).groupby("origin").apply(lambda x : x.fillna(method="ffill"))

/var/folders/h1/6sllxx6j0zb6l_hvj_9ys3qh0000gn/T/ipykernel_10418/4238929852.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_weather = df_weather.sort_values(["origin", "time_hour"]).groupby("origin").apply(lambda x : x.fillna(method="ffill"))
/var/folders/h1/6sllxx6j0zb6l_hvj_9ys3qh0000gn/T/ipykernel_10418/4238929852.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weather = df_weather.sort_values(["origin", "time_hour"]).groupby("origin").apply(lambda x : x.fillna(method="ffill"))


In [66]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 26201 entries, ('EWR', 0) to ('LGA', 26200)
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   origin      26201 non-null  object 
 1   year        26201 non-null  float64
 2   month       26201 non-null  float64
 3   day         26201 non-null  int64  
 4   hour        26201 non-null  int64  
 5   wind_dir    26200 non-null  float64
 6   wind_speed  26200 non-null  float64
 7   wind_gust   26200 non-null  float64
 8   visib       26201 non-null  float64
 9   time_hour   26201 non-null  object 
dtypes: float64(6), int64(2), object(2)
memory usage: 3.3+ MB


In [67]:
df_weather[8730:8740]

origin    year  month  day  hour  wind_dir  wind_speed  wind_gust  \
origin                                                                          
EWR    8730    EWR  2017.0   12.0   30    19     230.0     4.60312   5.297178   
       8731    EWR  2017.0   12.0   30    20     220.0     8.05546   9.270062   
       8732    EWR  2017.0   12.0   30    21     280.0    10.35702  11.918651   
       8733    EWR  2017.0   12.0   30    22     260.0     6.90468   7.945768   
       8734    EWR  2017.0   12.0   30    23     280.0     6.90468   7.945768   
JFK    8735    JFK  2017.0    1.0    1     0       NaN         NaN        NaN   
       8736    JFK  2017.0    1.0    1     1     220.0    14.96014  17.215830   
       8737    JFK  2017.0    1.0    1     2     200.0    14.96014  17.215830   
       8738    JFK  2017.0    1.0    1     3     220.0    13.80936  15.891535   
       8739    JFK  2017.0    1.0    1     4     220.0    12.65858  14.567241   

             visib            time_hour  
origin                                   
EWR    8730    4.0  2017-12-31 03:00:00  
       8731    9.0  2017-12-31 04:00:00  
       8732   10.0  2017-12-31 05:00:00  
       8733   10.0  2017-12-31 06:00:00  
       8734   10.0  2017-12-31 07:00:00  
JFK    8735   10.0  2017-01-01 08:00:00  
       8736   10.0  2017-01-01 09:00:00  
       8737   10.0  2017-01-01 10:00:00  
       8738   10.0  2017-01-01 11:00:00  
       8739   10.0  2017-01-01 12:00:00

In [68]:
df_weather[df_weather['wind_dir'].isnull()]

,,origin,year,month,day,hour,wind_dir,wind_speed,wind_gust,visib,time_hour
origin,,,,,,,,,,,
JFK,8735,JFK,2017.0,1.0,1,0,NaN,NaN,NaN,10.0,2017-01-01 08:00:00


In [69]:
df_weather = df_weather.apply(lambda x: x.bfill()).reset_index(drop=True)


In [70]:
df_weather[df_weather['wind_dir'].isnull()]

,origin,year,month,day,hour,wind_dir,wind_speed,wind_gust,visib,time_hour


In [71]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26201 entries, 0 to 26200
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   origin      26201 non-null  object 
 1   year        26201 non-null  float64
 2   month       26201 non-null  float64
 3   day         26201 non-null  int64  
 4   hour        26201 non-null  int64  
 5   wind_dir    26201 non-null  float64
 6   wind_speed  26201 non-null  float64
 7   wind_gust   26201 non-null  float64
 8   visib       26201 non-null  float64
 9   time_hour   26201 non-null  object 
dtypes: float64(6), int64(2), object(2)
memory usage: 2.0+ MB


### 1-3 Data Join
#### Join Process

#### 1. Join df_flights with df_airports: Join using df_flights["dest"] and df_airports["faa"]. Although it's possible to join using df_flights["origin"] and df_airports["faa"], the number of airports for origin is limited, so the join is performed on dest only.
    Combine flight information with detailed airport information.

#### 2. Join the result of step 1 with df_planes: Join using step1_df["tailnum"] and df_planes["tailnum"].
    Combine flight information with detailed plane information.

#### 3. Join the result of step 2 with df_weather: Join using step2_df[["origin", "time_hour"]] and df_weather[["origin", "time_hour"]].
    Combine flight information with weather information for the departure and arrival locations.

In [72]:
df_flight_and_airport = pd.merge(df_flights, df_airports, how='left', left_on='dest',right_on='faa')
df_flight_and_airport.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,minute,time_hour,faa,name,lat,lon,alt,tz,dst,tzone
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,...,14.0,2017-10-01 21:00:00,MIA,Miami International Airport,25.793200,-80.290604,8.0,-5.0,A,1.0
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,...,25.0,2017-10-01 05:00:00,IAH,George Bush Intercontinental Houston Airport,29.984400,-95.341400,97.0,-6.0,A,2.0
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,...,45.0,2017-10-01 05:00:00,MCO,Orlando International Airport,28.429399,-81.308998,96.0,-5.0,A,1.0
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,...,50.0,2017-10-01 05:00:00,ORD,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,-6.0,A,2.0
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,...,0.0,2017-10-01 06:00:00,FLL,Fort Lauderdale Hollywood International Airport,26.072599,-80.152702,9.0,-5.0,A,1.0


In [73]:
df_f_a_p = pd.merge(df_flight_and_airport, df_planes, how='left', on='tailnum', suffixes=("_flights","_planes"))
df_f_a_p.head()

,year_flights,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tz,dst,tzone,year_planes,type,manufacturer,model,engines,seats,engine
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,...,-5.0,A,1.0,1998.0,Fixed wing multi engine,BOEING,737-724,2.0,149.0,Turbo-fan
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,...,-6.0,A,2.0,2004.0,Fixed wing multi engine,BOEING,737-824,2.0,149.0,Turbo-fan
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,...,-5.0,A,1.0,2002.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,Turbo-fan
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,...,-6.0,A,2.0,2012.0,Fixed wing multi engine,BOEING,737-924ER,2.0,191.0,Turbo-fan
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,...,-5.0,A,1.0,2017.0,Fixed wing multi engine,AIRBUS,A321-231,2.0,379.0,Turbo-fan


In [74]:
# Join the result of step 2 with df_weather
# Two types of joins are possible:
# 1) Weather information at the departure time
# 2) Weather information at the arrival time: It's unclear how to obtain this with the given data, so it is not performed. However, if possible, it would be good to include.

# 3 Weather information at the departure time
df_f_a_p["dest_datetime"] = pd.to_datetime(df_f_a_p["time_hour"])
# Since time_hour reflects the scheduled departure time, the actual delayed departure time needs to be added to this data.
# Additionally, time_hour does not include 'minutes', so the minutes column data needs to be added as well.
df_f_a_p["dest_datetime"] = df_f_a_p.apply(lambda x: x.dest_datetime + pd.Timedelta(minutes=x.dep_delay) + pd.Timedelta(minutes=x.minute), axis=1)


In [75]:
df_f_a_p["time_hour"]

0         2017-10-01 21:00:00
1         2017-10-01 05:00:00
2         2017-10-01 05:00:00
3         2017-10-01 05:00:00
4         2017-10-01 06:00:00
                 ...         
294891    2017-09-30 22:00:00
294892    2017-09-30 22:00:00
294893    2017-09-30 22:00:00
294894    2017-09-30 23:00:00
294895    2017-09-30 22:00:00
Name: time_hour, Length: 294896, dtype: object

In [76]:
df_f_a_p["dest_datetime"]

0        2017-10-02 00:27:00
1        2017-10-01 05:19:00
2        2017-10-01 05:44:00
3        2017-10-01 05:46:00
4        2017-10-01 05:52:00
                 ...        
294891   2017-09-30 22:38:00
294892   2017-09-30 22:41:00
294893   2017-09-30 22:48:00
294894   2017-09-30 23:19:00
294895   2017-09-30 23:25:00
Name: dest_datetime, Length: 294896, dtype: datetime64[ns]

In [77]:
df_f_a_p.head()

,year_flights,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,dst,tzone,year_planes,type,manufacturer,model,engines,seats,engine,dest_datetime
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,...,A,1.0,1998.0,Fixed wing multi engine,BOEING,737-724,2.0,149.0,Turbo-fan,2017-10-02 00:27:00
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,...,A,2.0,2004.0,Fixed wing multi engine,BOEING,737-824,2.0,149.0,Turbo-fan,2017-10-01 05:19:00
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,...,A,1.0,2002.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,Turbo-fan,2017-10-01 05:44:00
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,...,A,2.0,2012.0,Fixed wing multi engine,BOEING,737-924ER,2.0,191.0,Turbo-fan,2017-10-01 05:46:00
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,...,A,1.0,2017.0,Fixed wing multi engine,AIRBUS,A321-231,2.0,379.0,Turbo-fan,2017-10-01 05:52:00


In [78]:
df_weather.head()

,origin,year,month,day,hour,wind_dir,wind_speed,wind_gust,visib,time_hour
0,EWR,2017.0,1.0,1,0,200.0,17.26170,19.864419,10.0,2017-01-01 08:00:00
1,EWR,2017.0,1.0,1,1,220.0,16.11092,18.540125,10.0,2017-01-01 09:00:00
2,EWR,2017.0,1.0,1,2,190.0,11.50780,13.242946,10.0,2017-01-01 10:00:00
3,EWR,2017.0,1.0,1,3,190.0,8.05546,9.270062,10.0,2017-01-01 11:00:00
4,EWR,2017.0,1.0,1,4,220.0,9.20624,10.594357,10.0,2017-01-01 12:00:00


In [79]:
# Remove minute information from dest_datetime to join with weather data
df_f_a_p["dest_datetime"] = df_f_a_p["dest_datetime"].dt.strftime("%Y-%m-%d %H:00:00")

# Display the first row to verify the changes
df_f_a_p.head()


,year_flights,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,dst,tzone,year_planes,type,manufacturer,model,engines,seats,engine,dest_datetime
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,...,A,1.0,1998.0,Fixed wing multi engine,BOEING,737-724,2.0,149.0,Turbo-fan,2017-10-02 00:00:00
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,...,A,2.0,2004.0,Fixed wing multi engine,BOEING,737-824,2.0,149.0,Turbo-fan,2017-10-01 05:00:00
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,...,A,1.0,2002.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,Turbo-fan,2017-10-01 05:00:00
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,...,A,2.0,2012.0,Fixed wing multi engine,BOEING,737-924ER,2.0,191.0,Turbo-fan,2017-10-01 05:00:00
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,...,A,1.0,2017.0,Fixed wing multi engine,AIRBUS,A321-231,2.0,379.0,Turbo-fan,2017-10-01 05:00:00


In [86]:
df_f_a_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294896 entries, 0 to 294895
Data columns (total 35 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year_flights    294896 non-null  int64  
 1   month           294896 non-null  int64  
 2   day             294896 non-null  int64  
 3   dep_time        294896 non-null  float64
 4   sched_dep_time  294896 non-null  int64  
 5   dep_delay       294896 non-null  float64
 6   arr_time        294896 non-null  float64
 7   sched_arr_time  294896 non-null  int64  
 8   arr_delay       294896 non-null  float64
 9   carrier         294896 non-null  object 
 10  flight          294896 non-null  int64  
 11  tailnum         294896 non-null  object 
 12  origin          294896 non-null  object 
 13  dest            294896 non-null  object 
 14  air_time        294896 non-null  float64
 15  distance        294896 non-null  float64
 16  hour            294896 non-null  float64
 17  minute    

In [85]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26201 entries, 0 to 26200
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   origin      26201 non-null  object 
 1   year        26201 non-null  float64
 2   month       26201 non-null  float64
 3   day         26201 non-null  int64  
 4   hour        26201 non-null  int64  
 5   wind_dir    26201 non-null  float64
 6   wind_speed  26201 non-null  float64
 7   wind_gust   26201 non-null  float64
 8   visib       26201 non-null  float64
 9   time_hour   26201 non-null  object 
dtypes: float64(6), int64(2), object(2)
memory usage: 2.0+ MB


In [87]:
# To join, df_weather['time_hour'] should chnage into 'dest_datetime'
df_weather = df_weather.rename(columns={'time_hour': 'dest_datetime' })

In [88]:
df_final = pd.merge(df_f_a_p, df_weather, how="left", on=["origin","dest_datetime"], suffixes=("", "_weather"))
# _x, _y

In [89]:
df_final.head()

,year_flights,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,engine,dest_datetime,year,month_weather,day_weather,hour_weather,wind_dir,wind_speed,wind_gust,visib
0,2017,10,1,27.0,2114,193.0,317.0,15,182.0,UA,...,Turbo-fan,2017-10-02 00:00:00,2017.0,10.0,1.0,17.0,40.0,9.20624,10.594357,10.0
1,2017,10,1,519.0,525,-6.0,741.0,759,-18.0,UA,...,Turbo-fan,2017-10-01 05:00:00,2017.0,9.0,30.0,22.0,360.0,16.11092,18.540125,10.0
2,2017,10,1,544.0,545,-1.0,815.0,819,-4.0,B6,...,Turbo-fan,2017-10-01 05:00:00,2017.0,9.0,30.0,22.0,360.0,16.11092,18.540125,10.0
3,2017,10,1,546.0,550,-4.0,702.0,719,-17.0,UA,...,Turbo-fan,2017-10-01 05:00:00,2017.0,9.0,30.0,22.0,360.0,19.56326,22.513008,10.0
4,2017,10,1,552.0,600,-8.0,846.0,850,-4.0,NK,...,Turbo-fan,2017-10-01 05:00:00,2017.0,9.0,30.0,22.0,360.0,16.11092,18.540125,10.0


### 1-4 Duplicated Data

In [90]:
print("Number of Duplicated :", len(df_final[df_final.duplicated()])) 

Number of Duplicated : 0


## 2. Data EDA

### 2-1 Data Type and Summary

In [91]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294896 entries, 0 to 294895
Data columns (total 43 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year_flights    294896 non-null  int64  
 1   month           294896 non-null  int64  
 2   day             294896 non-null  int64  
 3   dep_time        294896 non-null  float64
 4   sched_dep_time  294896 non-null  int64  
 5   dep_delay       294896 non-null  float64
 6   arr_time        294896 non-null  float64
 7   sched_arr_time  294896 non-null  int64  
 8   arr_delay       294896 non-null  float64
 9   carrier         294896 non-null  object 
 10  flight          294896 non-null  int64  
 11  tailnum         294896 non-null  object 
 12  origin          294896 non-null  object 
 13  dest            294896 non-null  object 
 14  air_time        294896 non-null  float64
 15  distance        294896 non-null  float64
 16  hour            294896 non-null  float64
 17  minute    

In [92]:
df_final.isna().sum()

year_flights          0
month                 0
day                   0
dep_time              0
sched_dep_time        0
dep_delay             0
arr_time              0
sched_arr_time        0
arr_delay             0
carrier               0
flight                0
tailnum               0
origin                0
dest                  0
air_time              0
distance              0
hour                  0
minute                0
time_hour             0
faa                5982
name               5982
lat                5982
lon                5982
alt                5982
tz                 5982
dst                5982
tzone              5982
year_planes       43853
type              36313
manufacturer      36313
model             36313
engines           36313
seats             36930
engine            36313
dest_datetime         0
year                783
month_weather       783
day_weather         783
hour_weather        783
wind_dir            783
wind_speed          783
wind_gust       

In [94]:
# categorical and numerical
for column_name in list(df_final.columns):
    print(column_name, type(df_final[column_name][0]), df_final[column_name].unique())

year_flights <class 'numpy.int64'> [2017]
month <class 'numpy.int64'> [10 11 12  1  2  3  4  5  6  7  8  9]
day <class 'numpy.int64'> [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
dep_time <class 'numpy.float64'> [ 27. 519. 544. ... 254. 425. 400.]
sched_dep_time <class 'numpy.int64'> [2114  525  545 ... 2231 2111 2354]
dep_delay <class 'numpy.float64'> [ 1.930e+02 -6.000e+00 -1.000e+00 -4.000e+00 -8.000e+00 -7.000e+00
 -1.200e+01 -5.000e+00 -3.000e+00 -9.000e+00  7.330e+02  0.000e+00
  3.300e+01  1.000e+00 -1.100e+01 -1.000e+01 -2.000e+00  1.200e+01
 -1.400e+01  5.000e+00  3.000e+00  2.000e+00  1.500e+01  1.400e+01
  3.400e+01  5.200e+01  1.530e+02 -1.700e+01 -1.600e+01  7.000e+00
 -1.300e+01  2.200e+01  8.000e+00  6.100e+01 -2.100e+01  7.600e+01
  2.800e+01  4.000e+00  9.000e+00  1.020e+02  1.080e+02 -1.500e+01
  1.800e+01  5.000e+01  1.900e+01  6.000e+00  4.900e+01  2.000e+01
  1.000e+01  3.000e+01  2.300e+01  5.600e+01  1.600e+01  6

In [95]:
df_final.select_dtypes(include=['int64']).columns

Index(['year_flights', 'month', 'day', 'sched_dep_time', 'sched_arr_time',
       'flight'],
      dtype='object')

In [96]:
# If float type columns do not exhibit numeric characteristics, they are heuristically judged and classified.
df_final.select_dtypes(include=['float64']).columns

# dep_time and arr_time represent times, and since the hour and minute data are in datetime format (%H%M), conversion is needed.
# wind_dir is also considered categorical (because it represents an angle).


Index(['dep_time', 'dep_delay', 'arr_time', 'arr_delay', 'air_time',
       'distance', 'hour', 'minute', 'lat', 'lon', 'alt', 'tz', 'tzone',
       'year_planes', 'engines', 'seats', 'year', 'month_weather',
       'day_weather', 'hour_weather', 'wind_dir', 'wind_speed', 'wind_gust',
       'visib'],
      dtype='object')

In [97]:
df_final["dep_time"] = df_final["dep_time"].apply(lambda x: '{0:04d}'.format(int(x)))
df_final["arr_time"] = df_final["arr_time"].apply(lambda x: '{0:04d}'.format(int(x)))
df_final["wind_dir"] = df_final["wind_dir"].astype("str")

In [98]:
list_categorical_columns = list(df_final.select_dtypes(include=['int64', 'object']).columns)
list_numeric_columns = list(df_final.select_dtypes(include=['float64']).columns)
target_column = "arr_delay"

In [99]:
print(len(df_final.columns))
print(len(list_categorical_columns))
print(len(list_numeric_columns))

43
22
21


In [100]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294896 entries, 0 to 294895
Data columns (total 43 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year_flights    294896 non-null  int64  
 1   month           294896 non-null  int64  
 2   day             294896 non-null  int64  
 3   dep_time        294896 non-null  object 
 4   sched_dep_time  294896 non-null  int64  
 5   dep_delay       294896 non-null  float64
 6   arr_time        294896 non-null  object 
 7   sched_arr_time  294896 non-null  int64  
 8   arr_delay       294896 non-null  float64
 9   carrier         294896 non-null  object 
 10  flight          294896 non-null  int64  
 11  tailnum         294896 non-null  object 
 12  origin          294896 non-null  object 
 13  dest            294896 non-null  object 
 14  air_time        294896 non-null  float64
 15  distance        294896 non-null  float64
 16  hour            294896 non-null  float64
 17  minute    

#### Summary of Data
#### Number of variables : 43
    numeric variables : 21(target column)
    categorical varabiels : 22 
#### Number of observations : 294896
#### Duplicated Cells : 0


### 2-2 Dependent Data Explore